In [2]:
import cv2
import numpy as np
import torch
from torchvision import transforms
from model.lanenet.LaneNet import LaneNet
from PIL import Image

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())

True


In [3]:
def preprocess_frame(frame, transform):
    frame = Image.fromarray(frame)
    frame = transform(frame)
    return frame

def detect_lanes(binary_pred, min_pixels=100, roi_height=0.6):
    height, width = binary_pred.shape
    roi_y = int(height * (1 - roi_height))
    
    # ROI 설정
    roi = binary_pred[roi_y:, :]
    
    # 차선 후보 픽셀 찾기
    left_candidates = []
    right_candidates = []
    
    for y in range(roi.shape[0]):
        row = roi[y]
        left_pixels = np.where(row[:width//2] > 200)[0]
        right_pixels = np.where(row[width//2:] > 200)[0] + width//2
        
        if len(left_pixels) > 0:
            left_candidates.append((np.mean(left_pixels), y + roi_y))
        if len(right_pixels) > 0:
            right_candidates.append((np.mean(right_pixels), y + roi_y))
    
    # 차선 피팅
    left_lane = right_lane = None
    if len(left_candidates) > min_pixels:
        left_lane = np.polyfit([p[1] for p in left_candidates], [p[0] for p in left_candidates], 2)
    if len(right_candidates) > min_pixels:
        right_lane = np.polyfit([p[1] for p in right_candidates], [p[0] for p in right_candidates], 2)
    
    return left_lane, right_lane

def draw_lanes(frame, left_lane, right_lane):
    height, width, _ = frame.shape
    y = np.linspace(height // 2, height, num=10)
    
    if left_lane is not None:
        left_x = left_lane[0] * y**2 + left_lane[1] * y + left_lane[2]
        pts_left = np.array([np.transpose(np.vstack([left_x, y]))])
        cv2.polylines(frame, [pts_left.astype(np.int32)], isClosed=False, color=(255, 0, 0), thickness=5)
    
    if right_lane is not None:
        right_x = right_lane[0] * y**2 + right_lane[1] * y + right_lane[2]
        pts_right = np.array([np.transpose(np.vstack([right_x, y]))])
        cv2.polylines(frame, [pts_right.astype(np.int32)], isClosed=False, color=(0, 0, 255), thickness=5)
    
    return frame

In [4]:
def test_with_webcam():
    model_path = r'C:\Users\yth12\Dropbox\4. 기타 자료\해군 AI 특강\AI_Lecture\Day1\Deep-Learning\LaneNet\log\lanenet_DeepLabv3+_CrossEntrophy_epoch100_batchsize8.pth'
    resize_height, resize_width = 480, 640

    data_transform = transforms.Compose([
        transforms.Resize((resize_height, resize_width)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    model = LaneNet(arch='DeepLabv3+')
    state_dict = torch.load(model_path)
    model.load_state_dict(state_dict)
    model.eval()
    model.to(DEVICE)

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to capture image.")
            break

        input_img = preprocess_frame(frame, data_transform).to(DEVICE)
        input_img = torch.unsqueeze(input_img, dim=0)

        with torch.no_grad():
            outputs = model(input_img)

        binary_pred = torch.squeeze(outputs['binary_seg_pred']).to('cpu').numpy() * 255
        
        # 차선 검출
        left_lane, right_lane = detect_lanes(binary_pred)
        
        # 검출된 차선 그리기
        result_frame = draw_lanes(frame, left_lane, right_lane)

        cv2.imshow('Lane Detection Result', result_frame)
        cv2.imshow('Binary Segmentation', binary_pred.astype(np.uint8))

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [5]:
if __name__ == "__main__":
    test_with_webcam()

Use DeepLabv3+ as backbone


C:\Users\yth12\AppData\Local\Temp\ipykernel_4116\3336258325.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path)
